In [1]:
import pandas as pd
import glob
import os
import sqlite3
from collections import Counter

In [2]:
#https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

pd.set_option('mode.chained_assignment', None)

### Data source
https://premium.pff.com/ncaa/positions/2014/REGPO/receiving?division=fbs&position=WR

In [3]:
path = r'data' 
all_files = glob.glob(path + "/*.csv")
all_files

['data\\data.csv',
 'data\\labels.csv',
 'data\\ncaa_wr14.csv',
 'data\\ncaa_wr15.csv',
 'data\\ncaa_wr16.csv',
 'data\\ncaa_wr17.csv',
 'data\\ncaa_wr18.csv',
 'data\\ncaa_wr19.csv',
 'data\\ncaa_wr20.csv',
 'data\\ncaa_wr21.csv',
 'data\\ncaa_wr22.csv',
 'data\\ncaa_wr23.csv',
 'data\\nfl_wr15.csv',
 'data\\nfl_wr16.csv',
 'data\\nfl_wr17.csv',
 'data\\nfl_wr18.csv',
 'data\\nfl_wr19.csv',
 'data\\nfl_wr20.csv',
 'data\\nfl_wr21.csv',
 'data\\nfl_wr22.csv',
 'data\\nfl_wr23.csv']

In [4]:
#https://stackoverflow.com/questions/56949605/how-to-read-each-file-from-a-folder-and-create-seperate-data-frames-for-each-fil
d = {os.path.splitext(os.path.basename(f))[0] : pd.read_csv(f) for f in all_files} 

In [5]:
#create a list of tuples where each tuple is (name of csv,dataframe)

#https://stackoverflow.com/questions/10795973/python-dictionary-search-values-for-keys-using-regular-expression
ncaa_list = [(key, value) for key, value in d.items() if key.startswith("ncaa_")]

#add year column to each dataframe

for entry in ncaa_list:
    if 'year' not in entry[1].columns:
        name = entry[0][len(entry[0]) - 2:]
        year = "20"+name
        entry[1].insert(0, 'year', int(year))

#concat dfs in list into one df
ncaa_all = pd.DataFrame()
for i in ncaa_list:
    ncaa_all = pd.concat([ncaa_all,i[1]])

In [6]:
#do the above for NFL data

nfl_list = [(key, value) for key, value in d.items() if key.startswith("nfl_")]

for entry in nfl_list:
    if 'year' not in entry[1].columns:
        name = entry[0][len(entry[0]) - 2:]
        year = "20"+name
        entry[1].insert(0, 'year', int(year))

nfl_all = pd.DataFrame()
for i in nfl_list:
    nfl_all = pd.concat([nfl_all,i[1]])

In [7]:
nflWR = nfl_all.loc[nfl_all['position'] == 'WR']
ncaaWR = ncaa_all.loc[ncaa_all['position']  == 'WR']

In [8]:
len(ncaaWR.player_id.unique())

4804

In [9]:
len(ncaaWR.columns)

46

In [10]:
len(nflWR.player_id.unique())

630

In [11]:
#drop all columns that are not player and player_id
merged = pd.merge(ncaaWR, nflWR, left_on=['player','player_id'], right_on=['player','player_id'], how='inner')
columns_drop = list((Counter(list(merged.columns))-Counter(['player','player_id'])).elements())
merged.drop(columns=columns_drop, inplace=True)

#get the list of WRs who are in both the NCAA and NFL datasets
player_list = merged.drop_duplicates()

#Create separate dfs with WR players who have played in the NCCA and the NFL
ncaa = ncaaWR[ncaaWR['player_id'].isin(player_list['player_id'])]
nfl = nflWR[nflWR['player_id'].isin(player_list['player_id'])]

In [12]:
unique_ncaa = ncaa.drop_duplicates('player_id')
print(f"We have {len(unique_ncaa)} players who have played in both the NCAA between 2014-2022 and the NFL between 2015-2023")

We have 377 players who have played in both the NCAA between 2014-2022 and the NFL between 2015-2023


In [13]:
ncaa = ncaa.reset_index(drop=True)
nfl = nfl.reset_index(drop=True)

In [14]:
nfl.columns

Index(['year', 'player', 'player_id', 'position', 'team_name',
       'player_game_count', 'avg_depth_of_target', 'avoided_tackles',
       'caught_percent', 'contested_catch_rate', 'contested_receptions',
       'contested_targets', 'declined_penalties', 'drop_rate', 'drops',
       'first_downs', 'franchise_id', 'fumbles', 'grades_hands_drop',
       'grades_hands_fumble', 'grades_offense', 'grades_pass_block',
       'grades_pass_route', 'inline_rate', 'inline_snaps', 'interceptions',
       'longest', 'pass_block_rate', 'pass_blocks', 'pass_plays', 'penalties',
       'receptions', 'route_rate', 'routes', 'slot_rate', 'slot_snaps',
       'targeted_qb_rating', 'targets', 'touchdowns', 'wide_rate',
       'wide_snaps', 'yards', 'yards_after_catch',
       'yards_after_catch_per_reception', 'yards_per_reception', 'yprr'],
      dtype='object')

In [15]:
cols_keep = ['year', 'player', 'player_id','player_game_count', 'avg_depth_of_target', 'avoided_tackles','caught_percent','drops','grades_hands_drop',
       'grades_hands_fumble', 'grades_offense','grades_pass_route','route_rate', 'routes', 'slot_rate', 'slot_snaps',
       'targeted_qb_rating', 'targets', 'touchdowns', 'wide_rate','wide_snaps','routes','receptions', 'yards', 'yards_after_catch','yards_after_catch_per_reception', 'yards_per_reception', 'yprr']

In [16]:
cols_keep

['year',
 'player',
 'player_id',
 'player_game_count',
 'avg_depth_of_target',
 'avoided_tackles',
 'caught_percent',
 'drops',
 'grades_hands_drop',
 'grades_hands_fumble',
 'grades_offense',
 'grades_pass_route',
 'route_rate',
 'routes',
 'slot_rate',
 'slot_snaps',
 'targeted_qb_rating',
 'targets',
 'touchdowns',
 'wide_rate',
 'wide_snaps',
 'routes',
 'receptions',
 'yards',
 'yards_after_catch',
 'yards_after_catch_per_reception',
 'yards_per_reception',
 'yprr']

In [17]:
len(cols_keep)

28

In [18]:
nfl_filter = nfl[cols_keep]
groupby_NFL = nfl_filter.groupby('player_id')

In [19]:

groupby_ncaa = ncaa_filter.groupby('player_id')

In [20]:
#Replace all Nans with column means

final_season = groupby_ncaa.tail(1)
nan_cols = list(final_season.columns[final_season.isnull().any(axis=0)])
nanDF = final_season.loc[:,nan_cols].copy(deep=False)

for i in nan_cols:     #---Applying Only on variables with NaN values
    nanDF.loc[:,i].fillna(nanDF.loc[:,i].mean(), inplace=True)
    final_season[i] = nanDF[i]
    
final_season.isnull().values.any()

#reset index
final_season = final_season.reset_index(drop=True)

In [21]:
nan_cols

['grades_hands_drop', 'yards_after_catch_per_reception', 'yards_per_reception']

In [19]:
#NFL WRs who had less than 10 games played in their final college season

#final_season.loc[final_season['player_game_count'] < 10]

In [20]:
#final_season.describe()

In [22]:
#Create y-label DF
players = final_season[['player','player_id']]
players = pd.concat([players,pd.DataFrame(columns=list(('800_yard_season','1000_yard_season')))])
players.player_id = players.player_id.astype(int)

for idx,row in players.iterrows():
    
    player_id = row['player_id']
    player_history = nfl.loc[nfl['player_id'] == player_id]
    
    for idx, row in player_history.iterrows():
        if row['yards'] > 800:
            players.loc[players['player_id'] == player_id,'800_yard_season'] = 1
            
        if row['yards'] > 1000:
            players.loc[players['player_id'] == player_id,'1000_yard_season'] = 1
            
    if pd.isna(players.loc[players['player_id'] == player_id,'800_yard_season'].item()):
        players.loc[players['player_id'] == player_id,'800_yard_season'] = 0
        
    if pd.isna(players.loc[players['player_id'] == player_id,'1000_yard_season'].item()):
        players.loc[players['player_id'] == player_id,'1000_yard_season'] = 0

In [23]:
num_800 = len(players.loc[players['1000_yard_season'] == 1])

num_1000 = len(players.loc[players['800_yard_season'] == 1])

print(
f"The number of WR who has over 800 yards in a NFL season is {num_800} \n"
f"The number of WR who has over 1000 yards in a NFL season is {num_1000} \n"    
)

The number of WR who has over 800 yards in a NFL season is 41 
The number of WR who has over 1000 yards in a NFL season is 63 



In [26]:
data = final_season
labels = players

In [28]:
data.to_csv('data/data.csv', index=False)
labels.to_csv('data/labels.csv', index=False)